### MÓDULO: Sistemas de Aprendizaje Automático   

# Predicción del consumo de energía eléctrica con Redes LSTM y GRU: Un estudio comparativo.

## Justificación:

La predicción del consumo de energía eléctrica es crucial para la gestión eficiente de los recursos energéticos. En este ejercicio, aplicarás técnicas de aprendizaje profundo para predecir el consumo de energía eléctrica utilizando un conjunto de datos de series temporales.

## Tarea a realizar:

- Carga y explora el conjunto de datos ["Individual household electric power consumption dataset"](https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption) (disponible en UCI Machine Learning Repository). Este conjunto de datos contiene mediciones del consumo de energía eléctrica en un hogar durante un período de cuatro años.
- Realiza un preprocesamiento adecuado de los datos, incluyendo la limpieza de valores faltantes, la normalización y la preparación de los datos para su uso en modelos de series temporales.
- Implementa un modelo de Red LSTM para predecir el consumo de energía eléctrica.
- Implementa un modelo de Red GRU para realizar la misma tarea.
- Evalúa el rendimiento de ambos modelos utilizando métricas como el Error Cuadrático Medio (RMSE) y el Error Absoluto Medio (MAE).
- Compara los resultados obtenidos por los modelos LSTM y GRU, analizando sus fortalezas y debilidades en este contexto específico.

En definitiva, se debe elaborar un informe conciso, usando un cuaderno Jupyter, que resuma los hallazgos, incluyendo gráficos que visualicen las predicciones de ambos modelos y una tabla comparativa de las métricas de evaluación.

In [ ]:
#Gemini code assit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# --- 1. Carga y Exploración de Datos ---

# Cargar el dataset (descargado previamente y ubicado en la misma carpeta)
try:
    data = pd.read_csv('household_power_consumption.txt', sep=';', low_memory=False, na_values=['?'])
except FileNotFoundError:
    print("Error: El archivo 'household_power_consumption.txt' no se encuentra. Asegúrate de haberlo descargado y colocado en la misma carpeta que este script.")
    exit()

# Convertir la columna 'Date' a tipo datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Combinar 'Date' y 'Time' en una sola columna 'Datetime'
data['Datetime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].astype(str))
data.drop(['Date', 'Time'], axis=1, inplace=True)
data.set_index('Datetime', inplace=True)

# Exploración inicial
print("Información del dataset:")
print(data.info())
print("\nPrimeras filas:")
print(data.head())
print("\nEstadísticas descriptivas:")
print(data.describe())

# --- 2. Preprocesamiento de Datos ---

# 2.1. Manejo de Valores Faltantes
print("\nValores faltantes por columna antes del imputado:")
print(data.isnull().sum())

# Imputar valores faltantes con la media de cada columna
data.fillna(data.mean(), inplace=True)

print("\nValores faltantes por columna después del imputado:")
print(data.isnull().sum())

# 2.2. Normalización
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# 2.3. Preparación para Series Temporales
def create_dataset(dataset, look_back=60):
    """Crea un dataset para series temporales con 'look_back' pasos anteriores."""
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]  # Solo usamos 'Global_active_power'
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 60  # Usaremos los últimos 60 minutos para predecir el siguiente
X, y = create_dataset(data_scaled, look_back)

# Redimensionar para LSTM y GRU [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# 2.4. División en Entrenamiento y Prueba
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size, :], X[train_size:len(X), :]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

# --- 3. Modelo LSTM ---

# 3.1. Construcción del Modelo
model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(look_back, 1)))
model_lstm.add(Dense(1))

# 3.2. Compilación y Entrenamiento
model_lstm.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history_lstm = model_lstm.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=0)

# 3.3. Predicciones
y_pred_lstm = model_lstm.predict(X_test)

# --- 4. Modelo GRU ---

# 4.1. Construcción del Modelo
model_gru = Sequential()
model_gru.add(GRU(50, input_shape=(look_back, 1)))
model_gru.add(Dense(1))

# 4.2. Compilación y Entrenamiento
model_gru.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
history_gru = model_gru.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=0)

# 4.3. Predicciones
y_pred_gru = model_gru.predict(X_test)

# --- 5. Evaluación de Modelos ---

# 5.1. Invertir la Normalización
y_test_inv = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((len(y_test), data.shape[1]-1))), axis=1))[:,0]
y_pred_lstm_inv = scaler.inverse_transform(np.concatenate((y_pred_lstm.reshape(-1, 1), np.zeros((len(y_pred_lstm), data.shape[1]-1))), axis=1))[:,0]
y_pred_gru_inv = scaler.inverse_transform(np.concatenate((y_pred_gru.reshape(-1, 1), np.zeros((len(y_pred_gru), data.shape[1]-1))), axis=1))[:,0]

# 5.2. Calcular Métricas
rmse_lstm = np.sqrt(mean_squared_error(y_test_inv, y_pred_lstm_inv))
mae_lstm = mean_absolute_error(y_test_inv, y_pred_lstm_inv)
rmse_gru = np.sqrt(mean_squared_error(y_test_inv, y_pred_gru_inv))
mae_gru = mean_absolute_error(y_test_inv, y_pred_gru_inv)

print("\nMétricas de Evaluación:")
print(f"LSTM - RMSE: {rmse_lstm:.4f}, MAE: {mae_lstm:.4f}")
print(f"GRU - RMSE: {rmse_gru:.4f}, MAE: {mae_gru:.4f}")

# --- 6. Comparación y Visualización ---

# 6.1. Tabla Comparativa
comparison_table = pd.DataFrame({
    'Model': ['LSTM', 'GRU'],
    'RMSE': [rmse_lstm, rmse_gru],
    'MAE': [mae_lstm, mae_gru]
})
print("\nTabla Comparativa:")
print(comparison_table)

# 6.2. Gráficos de Predicciones
plt.figure(figsize=(14, 6))
plt.plot(y_test_inv, label='Real')
plt.plot(y_pred_lstm_inv, label='LSTM')
plt.plot(y_pred_gru_inv, label='GRU')
plt.title('Predicciones vs. Valores Reales')
plt.xlabel('Tiempo (minutos)')
plt.ylabel('Global_active_power')
plt.legend()
plt.show()

# 6.3. Gráficos de Pérdida
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['loss'], label='Train Loss')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss')
plt.title('LSTM - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_gru.history['loss'], label='Train Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('GRU - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# --- 7. Informe Conciso (Resumen) ---
print("\n--- Informe Conciso ---")
print("Resumen de Hallazgos:")
print("1. Se ha realizado la carga, exploración y preprocesamiento del dataset 'Individual household electric power consumption'.")
print("2. Se han imputado los valores faltantes con la media de cada columna.")
print("3. Se ha normalizado el dataset utilizando MinMaxScaler.")
print("4. Se han preparado los datos para series temporales, utilizando un 'look_back' de 60 minutos.")
print("5. Se han implementado y entrenado dos modelos: LSTM y GRU.")
print("6. Se han evaluado ambos modelos utilizando RMSE y MAE.")
print("7. Se han generado gráficos comparativos de las predicciones y las curvas de pérdida.")
print("\nComparación de Modelos:")
print(f"- **LSTM:** RMSE: {rmse_lstm:.4f}, MAE: {mae_lstm:.4f}")
print(f"- **GRU:** RMSE: {rmse_gru:.4f}, MAE: {mae_gru:.4f}")
print("\nAnálisis:")
if rmse_lstm < rmse_gru and mae_lstm < mae_gru:
    print("En este caso, el modelo LSTM ha mostrado un mejor rendimiento que el modelo GRU, con valores más bajos de RMSE y MAE.")
elif rmse_gru < rmse_lstm and mae_gru < mae_lstm:
    print("En este caso, el modelo GRU ha mostrado un mejor rendimiento que el modelo LSTM, con valores más bajos de RMSE y MAE.")
else:
    print("Los modelos LSTM y GRU han mostrado un rendimiento similar en este caso.")
print("\nConclusión:")
print("Ambos modelos, LSTM y GRU, son capaces de predecir el consumo de energía eléctrica con una precisión razonable. La elección entre uno u otro dependerá de los requisitos específicos del problema y de la importancia relativa de la velocidad de entrenamiento y la precisión.")
